In [1]:
# To disable `warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)`
import warnings
warnings.filterwarnings('ignore')

## 3. Atomic Functions

CVXPY uses the function information in this section and the [DCP rules](https://www.cvxpy.org/tutorial/dcp/index.html#dcp) to mark expressions with a __sign__ and __curvature__.

### Operators

The infix operators `+`, `-`, `*`, `/`, `@` are treated as functions.
- `+` and `-` are always __affine__ functions.
- `expr1 * expr2` is __affine__ when one of the expressions is constant.
- `expr1 / expr2` is __affine__ when `expr2` is a <u>scalar</u> constant.

Starting with Python 3.5, users can write `expr1 @ expr2` for matrix multiplication and dot products.
As of CVXPY version 1.1, we are adopting a new standard:

- `@` should be used for <u>matrix-matrix and matrix-vector multiplication</u>,
- `*` should be used for <u>matrix-scalar and vector-scalar multiplication</u>

Elementwise multiplication can be applied with the `cvxpy.multiply` function.

#### Indexing and slicing
Indexing in CVXPY follows <u>exactly the same semantics as NumPy ndarrays</u>. For example,

In [2]:
import cvxpy as cp

x = cp.Variable(5)

print(f"""
0 dimensional: {x[0].shape}
1 dimensional: {x[0:1].shape}
""")


0 dimensional: ()
1 dimensional: (1,)



#### Transpose
The transpose of any expression can be obtained using the syntax `expr.T`. Transpose is an __affine__ function.

#### Power
For any CVXPY expression `expr`, the power operator `expr**p` is equivalent to the function `cvxpy.power(expr, p)`.

### Scalar functions

A scalar function takes one or more scalars, vectors, or matrices as arguments and returns a scalar.
![](figures/dcp_scalar_functions.png)

#### Clarifications for scalar functions

For a vector expression `x`, `cvxpy.norm(x)` and `cvxpy.norm(x, 2)` give the __Euclidean norm__.

For a matrix expression `X`, however, `cvxpy.norm(X)` and `cvxpy.norm(X, 2)` give the __spectral norm__.

The function `cvxpy.norm(X, "fro")` is called the __Frobenius norm__ and `cvxpy.norm(X, "nuc")` the __nuclear norm__.
The nuclear norm can also be defined as the sum of `X`'s singular values.

The functions `cvxpy.max` and `cvxpy.min` give the largest and smallest entry, respectively, in a <u>single expression</u>.
Use `cvxpy.maximum` and `cvxpy.minimum` to find the max or min of a <u>list of scalar expressions</u>.

The CVXPY function `cvxpy.sum` sums all the entries in a <u>single expression</u>. The built-in Python `sum` should be used to add together a <u>list of expressions</u>. For example, the following code sums a list of three expressions:

```
expr_list = [expr1, expr2, expr3]
expr_sum = sum(expr_list)
```

### Functions along an axis

The functions `cvxpy.sum`, `cvxpy.norm`, `cvxpy.max`, `cvxpy.min`, `cvxpy.mean`, `cvxpy.std`, `cvxpy.var`, and `cvxpy.ptp` can be applied along an axis.

Given an `m` by `n` expression `expr`,
- `func(expr, axis=0, keepdims=True)` applies `func` to each column, returning a `1` by `n` expression.
- `func(expr, axis=1, keepdims=True)` applies `func` to each row, returning an `m` by `1` expression.

By default `keepdims=False`, which means dimensions of length 1 are dropped. For example, the following code sums along the columns and rows of a matrix variable:

In [3]:
X = cp.Variable((5, 4))
col_sums_keepdims = cp.sum(X, axis=0, keepdims=True) # Has size (1, 4)
col_sums = cp.sum(X, axis=0) # Has size (4,)
row_sums = cp.sum(X, axis=1) # Has size (5,)

col_sums_keepdims, col_sums, row_sums

(Expression(AFFINE, UNKNOWN, (1, 4)),
 Expression(AFFINE, UNKNOWN, (4,)),
 Expression(AFFINE, UNKNOWN, (5,)))

### Elementwise functions

These functions operate on each element of their arguments.
For example, if `X` is a `5` by `4` matrix variable, then `cvxpy.abs(X)` is a `5` by `4` matrix expression. `cvxpy.abs(X)[1, 2]` is equivalent to `cvxpy.abs(X[1, 2])`.

Elementwise functions that take multiple arguments, such as `cvxpy.maximum` and `cvxpy.multiply`, operate on the corresponding elements of each argument. For example, if `X` and `Y` are both `3` by `3` matrix variables, then `cvxpy.maximum(X, Y)` is a `3` by `3` matrix expression.

![](figures/dcp_elementwise_functions.png)

#### Clarifications on elementwise functions
The functions `cvxpy.log_normcdf` and `cvxpy.loggamma` are defined via approximations.
- `cvxpy.log_normcdf` has highest accuracy over the range -4 to 4.
- `cvxpy.loggamma` has similar accuracy over all positive reals.

### Vector/matrix functions

A vector/matrix function takes one or more scalars, vectors, or matrices as arguments and returns a vector or matrix.

CVXPY is conservative when it determines the sign of an Expression returned by one of these functions.
- If any argument to one of these functions has unknown sign, then the returned Expression will also have unknown sign.
- If all arguments have known sign but CVXPY can determine that the returned Expression would have different signs in different entries (for example, when stacking a positive Expression and a negative Expression) then the returned Expression will have unknown sign.

![](figures/dcp_vector_matrix_functions.png)

#### Clarifications on vector and matrix functions
The input to `cvxpy.bmat` is a list of lists of CVXPY expressions. It constructs a block matrix. The elements of each inner list are stacked horizontally and then the resulting block matrices are stacked vertically.

The output $y=\textbf{convolve}(c,x)$ has size $n+m-1$ and is defined as $y_k=\sum_{j=0}^k c[j]x[k-j]$.

The output $y=\textbf{vec}(X)$ is the matrix $X$ flattened in column-major order into a vector. Formally, $y_i=X_{i\mod m,\lfloor i/m\rfloor}$.

The output $Y=\textbf{reshape}(X,(m'n'),\texttt{order="F"})$ is the matrix $X$ cast into an $m'\times n'$ matrix. The entries are taken from $X$ in column-major order and stored in $Y$ in column-major order. Formally, $Y_{i,j}=\textbf{vec}(X)_{m',j+i}$. If `order="C"` then $X$ will be read in row-major order and $Y$ will be written to in row-major order.

The output $y=\textbf{upper\_tri}(X)$ is formed by concatenating partial rows of $X$. I.e., $y=(X[0,1\texttt{:}],X[1,2\texttt{:}],\dotsc,X[n-1,n])$.